# Create country JSON

In [249]:
import pandas as pd
import numpy as np
import os

# Get tables and define countries to exclude

base_table = pd.read_html('https://www.geonames.org/countries/', keep_default_na=False, encoding='utf-8')[1]
country_names_german = pd.read_html('https://www.laenderdaten.de/laender.aspx', keep_default_na=False, encoding='utf-8')[0][['Land', 'ISO-2']]
not_countries = pd.read_csv('not_countries.csv', sep=';')['countryCode'].tolist()
svg_not_existing = ['fm', 'mh', 'tv', 'xk']
countries_to_drop = not_countries + svg_not_existing

# Rename columns and clean up data

countries_table = base_table[['ISO-3166alpha2', 'Country', 'Capital', 'Continent']]
countries_table.rename(columns = {'ISO-3166alpha2':'countryCode', 'Country':'countryNameEn', 'Capital':'capital', 'Continent':'continent'}, inplace = True)
country_names_german.rename(columns = {'Land':'countryNameGer', 'ISO-2':'countryCode'}, inplace=True)
country_names_german = country_names_german[country_names_german['countryNameGer'] != 'Clipperton']

for index, row in country_names_german.iterrows():
    if not pd.isnull(row['countryCode']):
        country_names_german.at[index, 'countryCode'] = row['countryCode'].lower()
countries_table['filename'] = ''

countries_table['continent'] = countries_table['continent'].replace(
    to_replace=['EU', 'AS', 'OC', 'AF', 'NA', 'SA'],
    value=['Europa', 'Asia', 'Oceania', 'Africa', 'North America', 'South America'])

for index, row in countries_table.iterrows():
    countries_table.at[index, 'countryCode'] = row['countryCode'].lower()
    row['filename'] = row['countryCode'] + '.svg'
    if pd.isnull(row['continent']):
        countries_table.at[index, 'continent'] = 'North America'

countries_table = countries_table[~countries_table.countryCode.isin(countries_to_drop)]    

countries_table = countries_table.sort_values(by='countryCode')
countries_table.reset_index(inplace=True, drop=True)
countries_table

C:\Users\maxim\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\maxim\AppData\Local\Temp/ipykernel_12836/985183446.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries_table['filename'] = ''
C:\Users\maxim\AppData\Local\Temp/ipykernel_12836/985183446.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,countryCode,countryNameEn,capital,continent,filename
0,ad,Andorra,Andorra la Vella,Europa,ad.svg
1,ae,United Arab Emirates,Abu Dhabi,Asia,ae.svg
2,af,Afghanistan,Kabul,Asia,af.svg
3,ag,Antigua and Barbuda,St. John's,North America,ag.svg
4,al,Albania,Tirana,Europa,al.svg
...,...,...,...,...,...
187,ws,Samoa,Apia,Oceania,ws.svg
188,ye,Yemen,Sanaa,Asia,ye.svg
189,za,South Africa,Pretoria,Africa,za.svg
190,zm,Zambia,Lusaka,Africa,zm.svg


In [250]:
countries_table = pd.merge(countries_table, country_names_german, on = "countryCode", how = "outer")
countries_table = countries_table[['countryCode', 'countryNameEn', 'countryNameGer', 'capital', 'continent', 'filename']]
countries_table = countries_table.dropna()
# with open('countries.json', 'w', encoding='utf-8') as file:
#     countries_table.to_json(file, force_ascii=False)
countries_table.to_json(r'countries.json', orient='index', force_ascii=False)
countries_table

,countryCode,countryNameEn,countryNameGer,capital,continent,filename
0,ad,Andorra,Andorra,Andorra la Vella,Europa,ad.svg
1,ae,United Arab Emirates,Vereinigte Arabische Emirate,Abu Dhabi,Asia,ae.svg
2,af,Afghanistan,Afghanistan,Kabul,Asia,af.svg
3,ag,Antigua and Barbuda,Antigua und Barbuda,St. John's,North America,ag.svg
4,al,Albania,Albanien,Tirana,Europa,al.svg
...,...,...,...,...,...,...
187,ws,Samoa,Samoa,Apia,Oceania,ws.svg
188,ye,Yemen,Jemen,Sanaa,Asia,ye.svg
189,za,South Africa,Südafrika,Pretoria,Africa,za.svg
190,zm,Zambia,Sambia,Lusaka,Africa,zm.svg
